In [ ]:
import xml.etree.ElementTree as ET
from collections import defaultdict

# Đọc file XML
xml_file = 'vietnet_food (thủ công).xml'
tree = ET.parse(xml_file)
root = tree.getroot()

# Namespace
ns = {'dc': 'https://globalwordnet.github.io/schemas/dc/'}

# Dictionary lưu trữ thông tin
synsets = {}  # synset_id -> {lemmas: [], definition: str}
children = defaultdict(list)  # parent_synset_id -> [child_synset_ids]
parents = {}  # child_synset_id -> parent_synset_id

# Parse LexicalEntry để lấy lemmas
lemma_to_synset = defaultdict(list)
for entry in root.findall('.//LexicalEntry'):
    for sense in entry.findall('Sense'):
        synset_id = sense.get('synset')
        lemma = entry.find('Lemma').get('writtenForm')
        lemma_to_synset[synset_id].append(lemma)

# Parse Synsets
for synset in root.findall('.//Synset'):
    synset_id = synset.get('id')
    
    # Lấy definition
    definition_elem = synset.find('Definition')
    definition = definition_elem.text if definition_elem is not None else ""
    
    # Lấy lemmas
    lemmas = lemma_to_synset.get(synset_id, [])
    
    # Lưu thông tin synset
    synsets[synset_id] = {
        'lemmas': lemmas,
        'definition': definition
    }
    
    # Lấy quan hệ hypernym và hyponym
    for relation in synset.findall('SynsetRelation'):
        rel_type = relation.get('relType')
        target = relation.get('target')
        
        if rel_type == 'hypernym':
            parents[synset_id] = target
            children[target].append(synset_id)

# Tìm root nodes (không có hypernym)
root_synsets = [sid for sid in synsets.keys() if sid not in parents]

# Hàm đệ quy để xuất cây với định dạng yêu cầu
def export_tree(synset_id, level=0):
    lines = []
    
    # Lấy thông tin synset
    info = synsets[synset_id]
    lemmas = info['lemmas']
    definition = info['definition']
    
    # Tạo chuỗi lemmas
    lemmas_str = ', '.join(lemmas) if lemmas else ''
    
    # Tạo tab indent
    indent = '\t' * level
    
    # Tạo dòng xuất
    line = f"{indent}({level}) {{{lemmas_str}}} [{synset_id}]: {definition}"
    lines.append(line)
    
    # Đệ quy cho các con
    for child_id in sorted(children.get(synset_id, [])):
        lines.extend(export_tree(child_id, level + 1))
    
    return lines

# Xuất toàn bộ cây
output_lines = []
for root_id in sorted(root_synsets):
    output_lines.extend(export_tree(root_id))

# Lưu ra file
output_file = 'vietnet_food_export.txt'
with open(output_file, 'w', encoding='utf-8') as f:
    f.write('\n'.join(output_lines))

print(f"✅ Đã xuất {len(output_lines)} dòng ra file: {output_file}")
print(f"✅ Tổng số synsets: {len(synsets)}")
print(f"✅ Số root synsets: {len(root_synsets)}")
print("\n📄 10 dòng đầu tiên:")
print('\n'.join(output_lines[:10]))


✅ Đã xuất 590 dòng ra file: vietnet_food_export.txt
✅ Tổng số synsets: 590
✅ Số root synsets: 2

📄 10 dòng đầu tiên:
(0) {thức ăn, thức} [vietnet-food-00045016-n]: thứ, món, loại, nói chung [thường nói về đồ ăn uống]
	(1) {cao lương mĩ vị, cao lương mỹ vị, mĩ vị, mỹ vị} [vietnet-food-00005530-n]: món ăn ngon và quý [nói khái quát]
		(2) {cao lương} [vietnet-food-00005529-n]: thịt béo và gạo ngon; món ăn ngon [nói khái quát]
		(2) {hải vị} [vietnet-food-00019494-n]: Thức ăn quý, chế biến từ các sản phẩm lấy ở biển
		(2) {sơn hào} [vietnet-food-00040423-n]: thức ăn quý, chế biến bằng sản phẩm lấy ở rừng núi
	(1) {cỗ} [vietnet-food-00009756-n]: tất cả những món ăn bày thành mâm để cúng lễ và ăn uống, theo tục lệ
	(1) {cơm} [vietnet-food-00010368-n]: những thức làm thành một bữa ăn [nói tổng quát]
	(1) {cơm bữa} [vietnet-food-00010375-n]: cơm ăn mỗi bữa hằng ngày [thường không có gì đặc biệt]
	(1) {cơm cháo} [vietnet-food-00010378-n]: cái ăn thường ngày như cơm, cháo, v.v. [nói khái quát]
